## Plant Sensor API Exercise
You've been given a virtual sensor API module which you can import with (`import sensor_api`) that can monitor plants. 
The API provides three functions:

- `connect(sensor_id)`: Connects to a sensor (returns True/False)
- `disconnect(sensor_id)`: Disconnects from a sensor (returns True/False)  
- `send_message(message)`: Sends a command and receives a reading (returns float or None)

#### Message Format
Messages must be formatted as: `"SENSOR_ID:COMMAND"`

Available commands:
- `SOIL_HUMIDITY`: Get soil humidity (0-100%)
- `AIR_HUMIDITY`: Get air humidity (0-100%)
- `TEMPERATURE`: Get temperature in Celsius

#### Before
This is how working with the API would currently look:

```python
import sensor_api

sensor_id = "GREENHOUSE_01"
sensor_api.connect(sensor_id)

soil_humidity = float(sensor_api.send_message(f"{id}:SOIL_HUMIDITY"))
air_humidity = float(sensor_api.send_message(f"{id}:AIR_HUMIDITY"))
temperature = float(sensor_api.send_message(f"{id}:TEMPERATURE"))

print(f"\n=== Plant Sensor {sensor_id} Readings ===")
print(f"Soil Humidity: {soil_humidity}%")
print(f"Air Humidity: {air_humidity}%")
print(f"Temperature: {temperature}°C")
print("=" * 35)
sensor_api.disconnect(id)
```

#### Your Task
Create a `PlantSensor` class that:

1. Stores the sensor ID when created
2. Automatically connects when initialized
3. Provides easy-to-use methods for each measurement type
4. Properly disconnects when done
5. Displays all readings in a nice format



#### Desired result
```python
# This is how your class should work:
sensor = PlantSensor("PLANT_01")
soil = sensor.get_soil_humidity()
air = sensor.get_air_humidity()
temp = sensor.get_temperature()
sensor.display_readings()
sensor.disconnect()

### Solution

In [1]:
import sensor_api

class PlantSensor:
    def __init__(self, sensor_id):
        """Initialize the sensor with an ID and connect to it."""
        self.sensor_id = sensor_id
        self.connected = sensor_api.connect(sensor_id)
        
        # Store readings
        self.soil_humidity = None
        self.air_humidity = None
        self.temperature = None
    
    def get_soil_humidity(self):
        """Get soil humidity reading."""
        if self.connected:
            message = f"{self.sensor_id}:SOIL_HUMIDITY"
            self.soil_humidity = sensor_api.send_message(message)
            return float(self.soil_humidity)
        return None
    
    def get_air_humidity(self):
        """Get air humidity reading."""
        if self.connected:
            message = f"{self.sensor_id}:AIR_HUMIDITY"
            self.air_humidity = sensor_api.send_message(message)
            return float(self.air_humidity)
        return None
    
    def get_temperature(self):
        """Get temperature reading."""
        if self.connected:
            message = f"{self.sensor_id}:TEMPERATURE"
            self.temperature = sensor_api.send_message(message)
            return float(self.temperature)
        return None
    
    def display_readings(self):
        """Display all sensor readings."""
        print(f"\n=== Plant Sensor {self.sensor_id} Readings ===")
        print(f"Soil Humidity: {self.soil_humidity}%")
        print(f"Air Humidity: {self.air_humidity}%")
        print(f"Temperature: {self.temperature}°C")
        print("=" * 35)
    
    def disconnect(self):
        """Disconnect from the sensor."""
        if self.connected:
            sensor_api.disconnect(self.sensor_id)
            self.connected = False

In [2]:
my_plant = PlantSensor("GREENHOUSE_01")

# Take readings
my_plant.get_soil_humidity()
my_plant.get_air_humidity()
my_plant.get_temperature()

# Display results
my_plant.display_readings()

# Clean up
my_plant.disconnect()

Sensor GREENHOUSE_01 connected successfully.

=== Plant Sensor GREENHOUSE_01 Readings ===
Soil Humidity: 22.8%
Air Humidity: 57.4%
Temperature: 17.3°C
Sensor GREENHOUSE_01 disconnected.
